<a href="https://colab.research.google.com/github/damvas/politica_internacional/blob/main/cronologia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pdfplumber
import pdfplumber
import re
import pandas as pd
import os

os.chdir('/content/drive/MyDrive/Python/Projetos/BD/Política Internacional')

In [ ]:
# jogo de acertar o ano de tal acontecimento
# adicionar parâmetro de range de anos (mais difícil, 0, não pode ser nem um ano a mais ou a menos)

In [ ]:
book_path = 'Cronologia das Relações Internacionais do Brasil.pdf'
text_path = 'cronologia_raw.txt'

In [ ]:
def process_text():
  text = ""
  with pdfplumber.open(book_path) as pdf:
      for i in range(9, 253):
        page = pdf.pages[i]
        text += ' ' + (page.extract_text() or '')
  return text

with open(text_path, 'r', encoding = 'utf-8') as file:
  text = file.read()

text = text.replace(' 10\nCRONOLOGIA\n','')
rows = text.split('.\n')
pattern = r'\.\s\d{2,3}\n'
new_rows = []
for row in rows:
  new_rows += re.split(pattern, row)

df = pd.DataFrame(data = new_rows, columns = ['desc'])
df['ano'] = df['desc'].apply(lambda x: re.findall(r'^(\d{4}) ', x)[0] if re.match(r'^\d{4} ', x) else None)
df['desc'] = df['desc'].str.replace(r'^\d{4} ', '', regex=True)
df['ano'] = df['ano'].fillna(method='ffill')
df = df.iloc[:len(df)-1,:]

In [ ]:
df

,desc,ano
0,"Buscando a rota para as Índias, o navegador ge...",1492
1,"O papa Alexandre VI, em sua bula Inter Coetera...",1493
2,"O Tratado de Tordesilhas, ou Capitulação da Pa...",1494
3,O navegador português Vasco da Gama chega a Ca...,1498
4,"Duarte Pacheco Pereira, sob instruções secreta...",1498
...,...,...
3104,Em reunião do Cone Sul sobre segurança nas fro...,2016
3105,Cai na Colômbia avião boliviano que transporta...,2016
3106,Promulgada emenda constitucional que limita os...,2016
3107,Resolução do Conselho de Segurança da ONU cond...,2016


In [ ]:
df

,desc,ano
0,"Buscando a rota para as Índias, o navegador ge...",1492
1,"O papa Alexandre VI, em sua bula Inter Coetera...",1493
2,"O Tratado de Tordesilhas, ou Capitulação da Pa...",1494
3,O navegador português Vasco da Gama chega a Ca...,1498
4,"Duarte Pacheco Pereira, sob instruções secreta...",1498
...,...,...
3104,Em reunião do Cone Sul sobre segurança nas fro...,2016
3105,Cai na Colômbia avião boliviano que transporta...,2016
3106,Promulgada emenda constitucional que limita os...,2016
3107,Resolução do Conselho de Segurança da ONU cond...,2016


In [ ]:
df

,desc,ano
0,"Buscando a rota para as Índias, o navegador ge...",1492
1,"O papa Alexandre VI, em sua bula Inter Coetera...",1493
2,"O Tratado de Tordesilhas, ou Capitulação da Pa...",1494
3,O navegador português Vasco da Gama chega a Ca...,1498
4,"Duarte Pacheco Pereira, sob instruções secreta...",1498
...,...,...
3104,Em reunião do Cone Sul sobre segurança nas fro...,2016
3105,Cai na Colômbia avião boliviano que transporta...,2016
3106,Promulgada emenda constitucional que limita os...,2016
3107,Resolução do Conselho de Segurança da ONU cond...,2016


In [ ]:
s = 'aaa'
re.split('a', s)

['', '', '', '']

In [ ]:
new_rows[:10]

['1492 Buscando a rota para as Índias, o navegador genovês Cristóvão Colombo, a serviço do Reino de\nCastela, chega ao continente americano e aporta na Ilha de San Salvador, nas Bahamas (12 out). O\ndescobrimento da América, marco das grandes navegações, abre uma disputa entre as duas\npotências que então lideravam as explorações ultramarinas: Espanha e Portugal',
 '1493 O papa Alexandre VI, em sua bula Inter Coetera, dada em Roma, concede aos reis católicos da\nEspanha, Isabel de Castela e Fernando de Aragão, "para a propagação da fé cristã", todas as terras\ndo Novo Mundo a oeste de um meridiano localizado a 100 léguas a oeste do arquipélago de Cabo\nVerde (4 maio). Portugal rejeita a bula por entender que não haveria novas terras a serem\ndescobertas na parte que lhe caberia na "divisão do mundo"',
 '1494 O Tratado de Tordesilhas, ou Capitulação da Partição do Mar Oceano, substitui a arbitragem\npontifícia pela negociação direta entre D. João II de Portugal e os reis espanhóis, Isab